In [3]:
import pandas as pd
from pyBKT.models import Model
from tqdm import tqdm

In [ ]:
def read_and_preprocess(dataset_name):
    if dataset_name=="HamptonAlg":
        df = pd.read_csv("data/raw_data/HamptonAlg_processed.csv")
        print(f"Number of rows before processing: {len(df)}")
        df = df[df['production'] != 'BLANK']  # Remove 'BLANK'
        print(f"Number of rows after : {len(df)}")
        production_mapping = {prod: idx for idx, prod in enumerate(df['production'].unique())}
        #print(df['production'].unique())
        df['knowledge'] = df['production'].map(production_mapping)
        pd.Series(production_mapping).to_csv('data/raw_data/HamptonAlg_production_mapping.csv')
        
        return df
    
    elif dataset_name=="Assistment_challenge":
        df = pd.read_csv("data/raw_data/assistant_competition.csv")
        print(f"Number of rows before processing: {len(df)}")
       
        print(f"Number of rows after : {len(df)}")
        skill_mapping = {prod: idx for idx, prod in enumerate(df['skill'].unique())}
        #print(df['skill'].unique())
        df['knowledge'] = df['skill'].map(skill_mapping)
        
       # df['skill'] = df['skill'].map(skill_mapping)
        pd.Series(skill_mapping).to_csv('data/raw_data/assistant_competition_skill_mapping.csv')
        
        return df
    else:
        print("no such dataset!")
        return

In [ ]:
def train_test_split(dataset_name,knowledge_column_name, student_column_name):
    
    test_df = pd.DataFrame()
    train_df = pd.DataFrame()
    
    df = read_and_preprocess(dataset_name)
    
    skill_count = 1
    
    for skill, skill_group in df.groupby(knowledge_column_name):
        print(f"Skill: {skill} — Total records: {len(skill_group)}")

        # Get all unique students for this skill
        unique_students = skill_group[student_column_name].unique()
        
        if len(unique_students)<5:
            continue

        # Sample 20% of the students
        n_sample = max(1, int(len(unique_students) * 0.2))  # ensure at least 1
        sampled_students = pd.Series(unique_students).sample(n=n_sample, random_state=42)

        # Split the skill_group based on whether student is in test or train
        test_part = skill_group[skill_group[student_column_name].isin(sampled_students)]
        train_part = skill_group[~skill_group[student_column_name].isin(sampled_students)]

        # Append to overall DataFrames
        test_df = pd.concat([test_df, test_part])
        train_df = pd.concat([train_df, train_part])

            

    # Step 3: Reset index (optional)
    test_df.reset_index(drop=True, inplace=True)
    train_df.reset_index(drop=True, inplace=True)
    
    print(f"Train set size: {len(train_df)}")
    print(f"Test set size: {len(test_df)}")
    
    # Step 5: Compare skills in train and test
    train_skills = set(train_df[knowledge_column_name].unique())
    test_skills = set(test_df[knowledge_column_name].unique())

    missing_skills = test_skills - train_skills

    if not missing_skills:
        print("✅ All skills in the test set are also present in the training set.")
        train_df.to_csv(dataset_name+ "_train.csv", index=False)
        test_df.to_csv(dataset_name +"_test.csv", index=False)
        
    else:
        print("❌ The following skills are in the test set but missing in the training set:")
        print(missing_skills)
        

In [ ]:
#train_test_split("HamptonAlg","knowledge","student")
#train_test_split("Assistment_challenge","knowledge","studentId")

In [5]:
from pyBKT.models import Model
from tqdm import tqdm

def run_BKT(dataset_name):
    
    if dataset_name in ["HamptonAlg","HamptonAlg_new"]:
        defaults = {'order_id': 'actionid','user_id': 'student', 'skill_name': 'knowledge', 'correct': 'assessment'}
    elif dataset_name in ["Assistment_challenge", "Assistment_challenge_new"]:
        defaults = {'order_id': 'action_num','user_id': 'studentId', 'skill_name': 'knowledge', 'correct': 'correct'}
    else:
        return
    
    train_path, test_path = dataset_name+ "_train.csv", dataset_name +"_test.csv"
    
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    train_df['knowledge'] = train_df['knowledge'].astype(str)
    test_df['knowledge'] = test_df['knowledge'].astype(str)

    # Identify skills present in the test set
    test_skills = test_df['knowledge'].unique()
    
    
    
    print()

    # To store all results
    all_results = []
    all_preds = []
    all_params = []
   
    
    # Initialize the model with an optional seed
    model = Model(seed = 42, num_fits = 1)
    
    # Crossvalidate with 5 folds on all skills in the CT dataset.
    #crossvalidated_errors = model.crossvalidate(data_path = train, folds = 5,defaults = defaults)
    
    #model.fit(data_path = train_path, defaults = defaults)
    
    #model.evaluate(data_path = test_path, metric = 'auc')
    
   
    
     #Loop through each skill and evaluate BKT
    for skill in tqdm(test_skills, desc="Fitting BKT per skill"):
        print("skill: ", skill)
        model = Model(seed=42, num_fits=1)

        # Filter data
        train_skill_df = train_df[train_df['knowledge'] == skill]
        test_skill_df = test_df[test_df['knowledge'] == skill]
        
        #print(train_skill_df)

#         if len(train_skill_df) < 10 or len(test_skill_df) < 5:
#             continue  # skip skills with too few records

        try:
            # Fit BKT on this skill
            model.fit(data=train_df, skills=skill, defaults=defaults)

            # Print parameters
            print(f"✅ Parameters for skill: {skill}")
            print(model.params())

        except Exception as e:
            print(f"❌ Error fitting BKT for skill '{skill}': {e}")
            continue


        # Evaluate on test set
        skill_result = model.evaluate(data=test_df,metric = 'auc')
        
        # Collect individual predictions (optional)
        preds = model.predict(data=test_df)
        preds['skill'] = skill
        all_preds.append(preds)
        
        print("skill_result: ", skill_result)

        # Save result for this skill
        all_results.append({
            'skill': skill,
            'auc': skill_result
        })
        all_params.append(model.params())
    
    

    # Save results to CSV
    results_df = pd.DataFrame(all_results)
    
    mean_auc = results_df['auc'].mean(skipna=True)
    print("mean_auc: ", mean_auc)
    
    results_df.to_json(f"{dataset_name}_BKT_results.json", orient='records', lines=True)
    
    all_params_df = pd.concat(all_params)

    # Step 2: Reset index to columns
    params_flat = all_params_df.reset_index()

    # Step 3: Filter for 'default' class only (optional, usually only one class)
    params_flat = params_flat[params_flat['class'] == 'default']

    # Step 4: Pivot to wide format
    params_pivot = params_flat.pivot(index='skill', columns='param', values='value').reset_index()

    # Step 5: Save to CSV
    params_pivot.to_csv(f"{dataset_name}_all_BKT_parameters_wide.csv", index=False)

    # Optional: display a few rows
    print(params_pivot.head())
    

In [110]:
run_BKT("HamptonAlg")

Fitting BKT per skill:   0%|                                                                    | 0/87 [00:00<?, ?it/s]

skill:  0
                        value
skill param   class          
0     prior   default 0.33847
      learns  default 0.01528
      guesses default 0.71148
      slips   default 0.07553
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/87 [00:12<18:17, 12.76s/it]

skill_result:  0.6965467239527391
skill:  1
                        value
skill param   class          
1     prior   default 0.54980
      learns  default 0.03219
      guesses default 0.45523
      slips   default 0.03412
      forgets default 0.00000


Fitting BKT per skill:   2%|█▍                                                          | 2/87 [00:26<18:50, 13.30s/it]

...

skill_result:  0.23076923076923078
skill:  85
                        value
skill param   class          
85    prior   default 0.88673
      learns  default 1.00000
      guesses default 0.02057
      slips   default 0.05696
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 86/87 [16:34<00:11, 11.93s/it]

skill_result:  0.5673076923076923
skill:  86
                        value
skill param   class          
86    prior   default 0.00084
      learns  default 0.19238
      guesses default 0.70419
      slips   default 0.00015
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 87/87 [17:39<00:00, 12.17s/it]

skill_result:  0.4074074074074074


In [3]:
run_BKT("HamptonAlg")

Fitting BKT per skill:   0%|                                                                    | 0/87 [00:00<?, ?it/s]

skill:  0
                        value
skill param   class          
0     prior   default 0.33847
      learns  default 0.01528
      guesses default 0.71148
      slips   default 0.07553
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/87 [00:16<23:14, 16.22s/it]

skill_result:  0.6965467239527391
skill:  1
                        value
skill param   class          
1     prior   default 0.54972
      learns  default 0.03219
      guesses default 0.45525
      slips   default 0.03411
      forgets default 0.00000


Fitting BKT per skill:   2%|█▍                                                          | 2/87 [00:26<18:08, 12.81s/it]

...

skill_result:  0.23076923076923078
skill:  85
                        value
skill param   class          
85    prior   default 0.79476
      learns  default 0.99663
      guesses default 0.41575
      slips   default 0.05688
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 86/87 [16:23<00:07,  7.74s/it]

skill_result:  0.5432692307692307
skill:  86
                        value
skill param   class          
86    prior   default 0.00103
      learns  default 0.19258
      guesses default 0.70405
      slips   default 0.00022
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 87/87 [17:20<00:00, 11.96s/it]

skill_result:  0.4074074074074074
mean_auc:  0.6367931055646762
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.71148 0.01528 0.33847 0.07553
1         1  0.00000  0.45525 0.03219 0.54972 0.03411
2        10  0.00000  0.28231 0.00918 0.52864 0.18225
3        11  0.00000  0.28008 0.08881 0.87062 0.10188
4        12  0.00000  0.23273 0.01248 0.32788 0.41369


In [12]:
run_BKT("Assistment_challenge")

Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.55998
      learns  default 0.23969
      guesses default 0.04399
      slips   default 0.55383
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:23<2:06:23, 83.33s/it]

skill_result:  0.5779043400305859
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.51784
      learns  default 0.04842
      guesses default 0.12526
      slips   default 0.62319
      forgets default 0.00000


Fitting BKT per skill:   2%|█▎                                                        | 2/92 [01:37<1:04:12, 42.81s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.80485
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [55:06<00:28, 14.03s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [55:15<00:00, 36.04s/it]

skill_result:  0.625
skill:  100
mean_auc:  0.5725632447419264
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.04399 0.23969 0.55998 0.55383
1         1  0.00000  0.12526 0.04842 0.51784 0.62319
2        10  0.00000  0.34152 0.02429 0.27142 0.31122
3        11  0.00000  0.16068 0.24051 0.38386 0.31949
4        12  0.00000  0.00044 0.64021 0.51832 0.65746


In [45]:


def run_BKT_poison(dataset_name, poison_mode):
    
    if dataset_name=="HamptonAlg":
        defaults = {'order_id': 'actionid','user_id': 'student', 'skill_name': 'knowledge', 'correct': 'assessment'}
    elif dataset_name=="Assistment":
        defaults = {'order_id': 'action_num','user_id': 'studentId', 'skill_name': 'knowledge', 'correct': 'correct'}
    else:
        return
    
    train_path, test_path = dataset_name+ "_poisoned_"+str(poison_mode)+".csv", dataset_name +"_test.csv"
    
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    print(train_df['knowledge'].isna().sum())
    
    train_df = train_df.dropna(subset=['knowledge'])
    
    train_df['knowledge'] = train_df['knowledge'].astype(int).astype(str) 
    test_df['knowledge']  = test_df['knowledge'].astype(int).astype(str)

    # Identify skills present in the test set
    test_skills = test_df['knowledge'].unique()
    

    # To store all results
    all_results = []
    all_preds = []
    all_params = []
   
    
    # Initialize the model with an optional seed
    model = Model(seed = 42, num_fits = 1)
    
    # Crossvalidate with 5 folds on all skills in the CT dataset.
    #crossvalidated_errors = model.crossvalidate(data_path = train, folds = 5,defaults = defaults)
    
    #model.fit(data_path = train_path, defaults = defaults)
    
    #model.evaluate(data_path = test_path, metric = 'auc')
    
   
    
     #Loop through each skill and evaluate BKT
    for skill in tqdm(test_skills, desc="Fitting BKT per skill"):
        print("skill: ", skill)
        model = Model(seed=42, num_fits=1)

        # Filter data
        train_skill_df = train_df[train_df['knowledge'] == skill]
        test_skill_df = test_df[test_df['knowledge'] == skill]
        
        print(train_skill_df['correct'].unique())

        if len(train_skill_df) < 10 or len(test_skill_df) < 5:
            print("len(train_skill_df) < 10 or len(test_skill_df) < 5")
            continue  # skip skills with too few records

        try:
            # Fit BKT on this skill
            model.fit(data=train_df, skills=skill, defaults=defaults)

            # Print parameters
            print(f"✅ Parameters for skill: {skill}")
            print(model.params())

        except Exception as e:
            print(f"❌ Error fitting BKT for skill '{skill}': {e}")
            continue


        # Evaluate on test set
        skill_result = model.evaluate(data=test_df,metric = 'auc')
        
        # Collect individual predictions (optional)
        preds = model.predict(data=test_df)
        preds['skill'] = skill
        all_preds.append(preds)
        
        print("skill_result: ", skill_result)

        # Save result for this skill
        all_results.append({
            'skill': skill,
            'auc': skill_result
        })
        all_params.append(model.params())
    
    

    # Save results to CSV
    results_df = pd.DataFrame(all_results)
    
    mean_auc = results_df['auc'].mean(skipna=True)
    print("mean_auc: ", mean_auc)
    
    results_df.to_json(f"{dataset_name}_poisoned_{poison_mode}_BKT_results.json", orient='records', lines=True)
    
    all_params_df = pd.concat(all_params)

    # Step 2: Reset index to columns
    params_flat = all_params_df.reset_index()

    # Step 3: Filter for 'default' class only (optional, usually only one class)
    params_flat = params_flat[params_flat['class'] == 'default']

    # Step 4: Pivot to wide format
    params_pivot = params_flat.pivot(index='skill', columns='param', values='value').reset_index()

    # Step 5: Save to CSV
    params_pivot.to_csv(f"{dataset_name}_poisoned_{poison_mode}_all_BKT_parameters_wide.csv", index=False)

    # Optional: display a few rows
    print(params_pivot.head())

In [35]:
poison_modes =[5,25,50]
dataset_names = ["HamptonAlg","Assistment"]

for dataset_name in dataset_names:
    for poison_mode in poison_modes:
        run_BKT_poison(dataset_name, poison_mode)
    

1


Fitting BKT per skill:   0%|                                                                    | 0/87 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.41292
      learns  default 0.03021
      guesses default 0.64072
      slips   default 0.09416
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/87 [00:11<16:58, 11.84s/it]

skill_result:  0.6838292158968851
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.55549
      learns  default 0.03387
      guesses default 0.44287
      slips   default 0.03335
      forgets default 0.00000


...

len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  80
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  81
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  82
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  83
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  84
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  85
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  86
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  87
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  88
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  89


Fitting BKT per skill:  95%|███████████████████████████████████████████████████████▊   | 87/92 [06:01<00:00, 29.31it/s]

len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  91
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  92
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  93
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  94
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  95
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  96
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  97
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  98
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  99
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [06:01<00:00,  3.93s/it]

mean_auc:  0.5611020472816515
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.48503 0.04644 0.10947 0.47743
1        11  0.00000  0.36898 0.00728 0.60161 0.45118
2        12  0.00000  0.44689 0.29219 0.60750 0.47941
3         2  0.00000  0.52532 0.01779 0.12556 0.59741
4         3  0.00000  0.36756 0.06164 0.41403 0.37228


In [7]:
run_BKT_poison("Assistment", 5)
run_BKT_poison("Assistment", 25)
run_BKT_poison("Assistment", 50)

NameError: name 'run_BKT_poison' is not defined

In [16]:
def run_BKT_dpa(dataset_name, attack_type, poison_mode):
    
    if dataset_name in ["HamptonAlg","HamptonAlg_new"]:
        defaults = {'order_id': 'actionid','user_id': 'student', 'skill_name': 'knowledge', 'correct': 'assessment'}
    elif dataset_name in ["Assistment_challenge", "Assistment_challenge_new","Assistment_new"]:
        defaults = {'order_id': 'action_num','user_id': 'studentId', 'skill_name': 'knowledge', 'correct': 'correct'}
    else:
        return
    
    train_path, test_path = dataset_name+ "_"+attack_type+"_" + str(poison_mode)+".csv", dataset_name +"_test.csv"
    
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    print(train_df['knowledge'].isna().sum())
    
    train_df = train_df.dropna(subset=['knowledge'])
    
    #train_df = train_df.dropna(subset=['correct'])
    if attack_type == "hint_abuse":
        train_df["correct"] = train_df["correct"].fillna(-1)
    
    train_df['knowledge'] = train_df['knowledge'].astype(int).astype(str) 
    test_df['knowledge']  = test_df['knowledge'].astype(int).astype(str)

    # Identify skills present in the test set
    test_skills = test_df['knowledge'].unique()
    

    # To store all results
    all_results = []
    all_preds = []
    all_params = []
   
    
    # Initialize the model with an optional seed
    model = Model(seed = 42, num_fits = 1)
    
    # Crossvalidate with 5 folds on all skills in the CT dataset.
    #crossvalidated_errors = model.crossvalidate(data_path = train, folds = 5,defaults = defaults)
    
    #model.fit(data_path = train_path, defaults = defaults)
    
    #model.evaluate(data_path = test_path, metric = 'auc')
    
   
    
     #Loop through each skill and evaluate BKT
    for skill in tqdm(test_skills, desc="Fitting BKT per skill"):
        print("skill: ", skill)
        model = Model(seed=42, num_fits=1)

        # Filter data
        train_skill_df = train_df[train_df['knowledge'] == skill]
        test_skill_df = test_df[test_df['knowledge'] == skill]
        
        
        
       # print(train_skill_df)

#         if len(train_skill_df) < 10 or len(test_skill_df) < 5:
#             print("len(train_skill_df) < 10 or len(test_skill_df) < 5")
#             continue  # skip skills with too few records

        try:
            # Fit BKT on this skill
            model.fit(data=train_df, skills=skill, defaults=defaults)

            # Print parameters
            print(f"✅ Parameters for skill: {skill}")
            print(model.params())

        except Exception as e:
            print(f"❌ Error fitting BKT for skill '{skill}': {e}")
            continue


        # Evaluate on test set
        skill_result = model.evaluate(data=test_df,metric = 'auc')
        
        # Collect individual predictions (optional)
        preds = model.predict(data=test_df)
        preds['skill'] = skill
        all_preds.append(preds)
        
        print("skill_result: ", skill_result)

        # Save result for this skill
        all_results.append({
            'skill': skill,
            'auc': skill_result
        })
        all_params.append(model.params())
    
    

    # Save results to CSV
    results_df = pd.DataFrame(all_results)
    
    mean_auc = results_df['auc'].mean(skipna=True)
    print("mean_auc: ", mean_auc)
    
    results_df.to_json(f"{dataset_name}_{attack_type}_poisoned_{poison_mode}_BKT_results.json", orient='records', lines=True)
    
    all_params_df = pd.concat(all_params)

    # Step 2: Reset index to columns
    params_flat = all_params_df.reset_index()

    # Step 3: Filter for 'default' class only (optional, usually only one class)
    params_flat = params_flat[params_flat['class'] == 'default']

    # Step 4: Pivot to wide format
    params_pivot = params_flat.pivot(index='skill', columns='param', values='value').reset_index()

    # Step 5: Save to CSV
    params_pivot.to_csv(f"{dataset_name}_{attack_type}_poisoned_{poison_mode}_all_BKT_parameters_wide.csv", index=False)

    # Optional: display a few rows
    print(params_pivot.head())

In [6]:
poison_modes =[5,25,50]
dataset_names = ["HamptonAlg"] #,"Assistment_sequential_pattern"

for dataset_name in dataset_names:
    for poison_mode in poison_modes:
        run_BKT_dpa(dataset_name, "sequential_pattern", poison_mode)

0


Fitting BKT per skill:   0%|                                                                    | 0/87 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.37390
      learns  default 0.01728
      guesses default 0.68785
      slips   default 0.08206
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/87 [00:13<18:46, 13.10s/it]

skill_result:  0.6930021482277121
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.55016
      learns  default 0.03218
      guesses default 0.45514
      slips   default 0.03418
      forgets default 0.00000


...

skill_result:  0.23076923076923078
skill:  85
✅ Parameters for skill: 85
                        value
skill param   class          
85    prior   default 0.85799
      learns  default 0.99905
      guesses default 0.19881
      slips   default 0.05694
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 86/87 [15:22<00:08,  8.22s/it]

skill_result:  0.5432692307692307
skill:  86
✅ Parameters for skill: 86
                        value
skill param   class          
86    prior   default 0.04974
      learns  default 0.11795
      guesses default 0.73749
      slips   default 0.00004
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 87/87 [16:05<00:00, 11.10s/it]

skill_result:  0.4074074074074074
mean_auc:  0.6396421724060919
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.71853 0.01438 0.30153 0.07474
1         1  0.00000  0.45523 0.03219 0.54983 0.03413
2        10  0.00000  0.28244 0.00918 0.52817 0.18213
3        11  0.00000  0.27628 0.08958 0.87151 0.10197
4        12  0.00000  0.23288 0.01249 0.32661 0.41337


In [ ]:
poison_modes =[5,25,50]
dataset_names = ["HamptonAlg"] #,"Assistment_sequential_pattern"

for dataset_name in dataset_names:
    for poison_mode in poison_modes:
        run_BKT_dpa(dataset_name, "sequential_pattern", poison_mode)

In [8]:
poison_modes =[5,25,50]
dataset_names = ["Assistment"]

for dataset_name in dataset_names:
    for poison_mode in poison_modes:
        run_BKT_dpa(dataset_name, "sequential_pattern", poison_mode)

1


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.65112
      learns  default 0.35249
      guesses default 0.05963
      slips   default 0.61766
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:36<55:32, 36.62s/it]

skill_result:  0.5833359700469335
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.82663
      learns  default 0.03453
      guesses default 0.12065
      slips   default 0.71216
      forgets default 0.00000


...

skill_result:  0.5834203448821389
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.86695
      learns  default 0.05874
      guesses default 0.08466
      slips   default 0.70347
      forgets default 0.00000


Fitting BKT per skill:   2%|█▎                                                        | 2/92 [01:43<1:09:58, 46.65s/it]

skill_result:  0.5406331921758418
skill:  2


Fitting BKT per skill:   2%|█▎                                                        | 2/92 [02:31<1:53:52, 75.92s/it]


KeyboardInterrupt: 

In [11]:
for poison_mode in poison_modes:
    run_BKT_dpa("Assistment", "poisoned", poison_mode)

1


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.32338
      learns  default 0.16415
      guesses default 0.15348
      slips   default 0.44598
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:09<14:35,  9.63s/it]

skill_result:  0.5773348098929494
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.65128
      learns  default 0.01494
      guesses default 0.17645
      slips   default 0.65981
      forgets default 0.00000


...

len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  68
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  69
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  70
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  71
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  72
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  73
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  74
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  75
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  76
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  77
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  78
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  79
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  80
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  81
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  82


Fitting BKT per skill:  89%|████████████████████████████████████████████████████▌      | 82/92 [03:36<00:00, 16.02it/s]

len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  83
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  84
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  85
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  86
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  87
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  88
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  89
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  91
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  92
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  93
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  94
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  95
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  96
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  97
len(train_skill_df) < 10 or len(test_skill_df) < 5


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [03:36<00:00,  2.35s/it]

skill:  98
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  99
len(train_skill_df) < 10 or len(test_skill_df) < 5
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.4996564454108868
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.51353 0.03890 0.13552 0.55558
1         1  0.00000  0.54933 0.12965 0.70110 0.53539
2         2  0.00000  0.50271 0.03422 0.20790 0.50872
3         3  0.00000  0.41033 0.37461 0.79109 0.48695
4         4  0.00000  0.52211 0.12830 0.18230 0.58124


In [21]:
def compare_skill_num():
    train_raw = "Assistment_challenge_train.csv"
    train_re = "Assistment_poisoned_5.csv"
    train_sq = "Assistment_sequential_pattern_5.csv"
    train_ha="Assistment_hint_abuse_5.csv"
    
    test_raw = "Assistment_test.csv"
    
    
    train_df_raw = pd.read_csv(train_raw)
    train_df_re = pd.read_csv(train_re)
    train_df_seq = pd.read_csv(train_sq)
    train_df_ha = pd.read_csv(train_ha)
    
    test_df = pd.read_csv(test_raw)
    
    train_df_raw = train_df_raw.dropna(subset=['knowledge'])
    train_df_re = train_df_re.dropna(subset=['knowledge'])
    train_df_seq = train_df_seq.dropna(subset=['knowledge'])
    train_df_ha = train_df_ha.dropna(subset=['knowledge'])
    
    
    train_df_raw['knowledge'] = train_df_raw['knowledge'].astype(int).astype(str)
    train_df_re['knowledge'] = train_df_re['knowledge'].astype(int).astype(str) 
    train_df_seq['knowledge'] = train_df_seq['knowledge'].astype(int).astype(str) 
    
    test_df['knowledge']  = test_df['knowledge'].astype(int).astype(str)

    # Identify skills present in the test set
    test_skills = test_df['knowledge'].unique()
    
    print(train_df_raw['knowledge'].isna().sum())
   

    
     #Loop through each skill and evaluate BKT
    for skill in tqdm(test_skills, desc="Fitting BKT per skill"):
        print("skill: ", skill)
        # Filter data
        train_skill_df_raw = train_df_raw[train_df_raw['knowledge'] == skill]
        train_skill_df_re = train_df_re[train_df_re['knowledge'] == skill]
        train_skill_df_seq = train_df_seq[train_df_seq['knowledge'] == skill]
        
        test_skill_df = test_df[test_df['knowledge'] == skill]
        
        if len(train_skill_df_re) < 10 or len(test_skill_df) < 5:
            print("len(train_skill_df_raw): ", len(train_skill_df_raw))
            print("len(train_skill_df_re): ", len(train_skill_df_re))
            print("len(train_skill_df_seq): ", len(train_skill_df_seq))
        
compare_skill_num()   

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0


Fitting BKT per skill:   2%|█▎                                                          | 2/92 [00:00<00:32,  2.77it/s]

...

skill:  96
skill:  97


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [00:14<00:00,  7.77it/s]

skill:  98
skill:  99


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [00:14<00:00,  6.30it/s]

skill:  100
len(train_skill_df_raw):  21
len(train_skill_df_re):  21
len(train_skill_df_seq):  21


In [3]:
run_BKT_dpa("Assistment", "poisoned", 5)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.20646
      learns  default 0.11966
      guesses default 0.18069
      slips   default 0.35030
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:09<13:52,  9.15s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.99940
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [45:21<00:28, 14.17s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.73115
      learns  default 0.00553
      guesses default 0.08153
      slips   default 0.89281
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [45:34<00:00, 29.73s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5721960731576687
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.18069 0.11966 0.20646 0.35030
1         1  0.00000  0.12249 0.05762 0.70357 0.67274
2        10  0.00000  0.32636 0.02572 0.42971 0.40489
3        11  0.00000  0.24008 0.20353 0.25567 0.25895
4        12  0.00000  0.00922 0.64778 0.56082 0.63935


In [4]:
run_BKT_dpa("Assistment", "sequential_pattern", 5)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.47225
      learns  default 0.16664
      guesses default 0.09553
      slips   default 0.51874
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:08<12:44,  8.40s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.99917
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [48:25<00:26, 13.11s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26929
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [48:31<00:00, 31.64s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5731524233096901
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.09553 0.16664 0.47225 0.51874
1         1  0.00000  0.14129 0.03423 0.45349 0.59915
2        10  0.00000  0.31683 0.01788 0.45124 0.40191
3        11  0.00000  0.15972 0.24136 0.38603 0.32062
4        12  0.00000  0.00438 0.62398 0.47784 0.65038


In [8]:
run_BKT_dpa("Assistment", "poisoned", 25)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.79547
      learns  default 0.36666
      guesses default 0.16541
      slips   default 0.56313
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:23<36:16, 23.92s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.69128
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [46:09<00:30, 15.29s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.50035
      learns  default 0.00390
      guesses default 0.39633
      slips   default 0.59637
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [46:15<00:00, 30.17s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5669176693758119
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.16541 0.36666 0.79547 0.56313
1         1  0.00000  0.14075 0.01039 0.95459 0.61810
2        10  0.00000  0.41280 0.02479 0.14627 0.33161
3        11  0.00000  0.07908 0.57454 0.81935 0.49402
4        12  0.00000  0.31292 0.28739 0.10464 0.52174


In [9]:
run_BKT_dpa("Assistment", "sequential_pattern", 25)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.63783
      learns  default 0.29335
      guesses default 0.00714
      slips   default 0.58349
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:09<1:46:05, 69.95s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.80230
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [44:28<00:26, 13.35s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26943
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [44:35<00:00, 29.09s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5707617673152747
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.00714 0.29335 0.63783 0.58349
1         1  0.00000  0.13378 0.04030 0.48644 0.61196
2        10  0.00000  0.33631 0.01991 0.32017 0.33908
3        11  0.00000  0.15612 0.24457 0.39415 0.32480
4        12  0.00000  0.00602 0.64609 0.49591 0.65188


In [10]:
run_BKT_dpa("Assistment", "sequential_pattern", 50)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.10264
      learns  default 0.04657
      guesses default 0.23478
      slips   default 0.18529
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:13<20:49, 13.73s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.51320
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [50:24<00:27, 13.88s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26936
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [50:30<00:00, 32.94s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5738820977368126
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.23478 0.04657 0.10264 0.18529
1         1  0.00000  0.04138 0.05711 0.97948 0.75241
2        10  0.00000  0.32537 0.01964 0.38930 0.37501
3        11  0.00000  0.12249 0.29622 0.48235 0.38324
4        12  0.00000  0.00471 0.62083 0.48337 0.65065


In [12]:
run_BKT_dpa("Assistment", "poisoned", 50)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.87351
      learns  default 0.10436
      guesses default 0.45422
      slips   default 0.50928
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/92 [00:17<27:01, 17.82s/it]

...

skill_result:  0.5
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.07825
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [32:56<00:22, 11.45s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.80838
      learns  default 0.08151
      guesses default 0.18281
      slips   default 0.79590
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [33:07<00:00, 21.60s/it]

skill_result:  0.375
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5247562346332769
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.45422 0.10436 0.87351 0.50928
1         1  0.00000  0.64268 0.08593 0.59491 0.54423
2        10  0.00000  0.43403 0.12794 0.62224 0.44867
3        11  0.00000  0.44127 0.13145 0.36617 0.41680
4        12  0.00000  0.45326 0.03137 0.36903 0.43342


In [21]:
run_BKT_dpa("Assistment", "hint_abuse", 5)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.61353
      learns  default 0.19982
      guesses default 0.04379
      slips   default 0.58374
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:19<2:00:13, 79.27s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.52240
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [53:10<00:30, 15.37s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [53:17<00:00, 34.75s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5722478099343098
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.04379 0.19982 0.61353 0.58374
1         1  0.00000  0.14072 0.03443 0.45728 0.60074
2        10  0.00000  0.22279 0.01240 0.80528 0.51874
3        11  0.00000  0.15480 0.24575 0.39711 0.32632
4        12  0.00000  0.00004 0.64929 0.51423 0.65764


In [17]:
train_ha="Assistment_hint_abuse_5.csv"
train_df_ha = pd.read_csv(train_ha)

print(train_df_ha["correct"].unique())

[ 0.  1. nan]


In [22]:
run_BKT_dpa("Assistment", "hint_abuse", 25)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.17999
      learns  default 0.02779
      guesses default 0.19599
      slips   default 0.35564
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:20<2:02:03, 80.48s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.92497
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [55:30<00:28, 14.15s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [55:37<00:00, 36.28s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5595367494918738
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.19599 0.02779 0.17999 0.35564
1         1  0.00000  0.14396 0.03224 0.44126 0.59416
2        10  0.00000  0.26541 0.01614 0.58132 0.47858
3        11  0.00000  0.27504 0.05681 0.15597 0.16758
4        12  0.00000  0.20602 0.00007 0.02271 0.41478


In [23]:
run_BKT_dpa("Assistment", "hint_abuse", 50)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.32893
      learns  default 0.00387
      guesses default 0.15843
      slips   default 0.51360
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:18<1:58:33, 78.17s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.99903
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [46:28<00:25, 12.81s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [46:36<00:00, 30.40s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5597040239025517
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.15843 0.00387 0.32893 0.51360
1         1  0.00000  0.13846 0.03385 0.62328 0.75059
2        10  0.00000  0.19929 0.00069 0.43921 0.40604
3        11  0.00000  0.22230 0.06829 0.23203 0.26123
4        12  0.00000  0.22373 0.02496 0.29380 0.77332


In [27]:
run_BKT_dpa("Assistment", "hint_abuse", 5)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.37349
      learns  default 0.09814
      guesses default 0.13362
      slips   default 0.46734
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:10<1:46:56, 70.51s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.97367
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [49:21<00:28, 14.36s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [49:33<00:00, 32.32s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5691616560749658
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.13362 0.09814 0.37349 0.46734
1         1  0.00000  0.13059 0.04322 0.49950 0.61694
2        10  0.00000  0.30681 0.01625 0.39760 0.38410
3        11  0.00000  0.23269 0.18052 0.22666 0.22280
4        12  0.00000  0.00446 0.62146 0.47778 0.65010


In [29]:
run_BKT_dpa("Assistment", "hint_abuse", 25)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.11934
      learns  default 0.02350
      guesses default 0.20820
      slips   default 0.22820
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:11<1:47:42, 71.01s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.44570
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [49:55<00:39, 19.52s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [50:02<00:00, 32.63s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5599753392603375
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.20820 0.02350 0.11934 0.22820
1         1  0.00000  0.12312 0.04955 0.53304 0.62924
2        10  0.00000  0.29031 0.01079 0.45029 0.42912
3        11  0.00000  0.26204 0.04776 0.23358 0.25052
4        12  0.00000  0.22962 0.03705 0.15302 0.82485


In [30]:
run_BKT_dpa("Assistment", "hint_abuse", 50)

c:\users\llii0118\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3473: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0


Fitting BKT per skill:   0%|                                                                    | 0/92 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.33282
      learns  default 0.00321
      guesses default 0.15805
      slips   default 0.51595
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                         | 1/92 [01:13<1:51:57, 73.82s/it]

...

skill_result:  0.472972972972973
skill:  98
✅ Parameters for skill: 98
                        value
skill param   class          
98    prior   default 0.33147
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000


Fitting BKT per skill:  98%|█████████████████████████████████████████████████████████▋ | 90/92 [49:16<00:27, 13.70s/it]

skill_result:  nan
skill:  99
✅ Parameters for skill: 99
                        value
skill param   class          
99    prior   default 0.26923
      learns  default 0.00000
      guesses default 0.00000
      slips   default 0.00000
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 92/92 [49:24<00:00, 32.22s/it]

skill_result:  0.625
skill:  100
len(train_skill_df) < 10 or len(test_skill_df) < 5
mean_auc:  0.5539154077512964
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.15805 0.00321 0.33282 0.51595
1         1  0.00000  0.15560 0.01317 0.32054 0.65443
2        10  0.00000  0.19465 0.00001 0.47520 0.42703
3        11  0.00000  0.22731 0.05643 0.24161 0.26799
4        12  0.00000  0.22651 0.02175 0.03136 0.78831


In [6]:
def check_student_overlap(dataset_name, student_column_name):
    train_file = f"{dataset_name}_train.csv"
    test_file = f"{dataset_name}_test.csv"
    
    # Read CSVs
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    
    # Extract student columns
    train_students = set(train_df[student_column_name].dropna().unique())
    test_students = set(test_df[student_column_name].dropna().unique())
    
    # Find overlap
    overlap = train_students & test_students
    
    unique_all = train_students | test_students
    
    print(f"Train unique students: {len(train_students)}")
    print(f"Test unique students: {len(test_students)}")
    print(f"Overlap students: {len(overlap)}")
    print(f"Overlap rate (vs train): {len(overlap)/len(train_students):.2%}")
    print(f"Overlap rate (vs test): {len(overlap)/len(test_students):.2%}")
    
    print(f"#Train: {len(train_df)}")
    print(f"#Test: {len(test_df)}")
    print(f"#unique_all: {len(unique_all)}")
    
check_student_overlap("HamptonAlg","student")    
check_student_overlap("Assistment_challenge","studentId")      


Train unique students: 59
Test unique students: 57
Overlap students: 57
Overlap rate (vs train): 96.61%
Overlap rate (vs test): 100.00%
#Train: 199070
#Test: 41167
#unique_all: 59
Train unique students: 1709
Test unique students: 1688
Overlap students: 1688
Overlap rate (vs train): 98.77%
Overlap rate (vs test): 100.00%
#Train: 754025
#Test: 188727
#unique_all: 1709


In [6]:
run_BKT("HamptonAlg_new")

Fitting BKT per skill:   0%|                                                                    | 0/86 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.38906
      learns  default 0.01967
      guesses default 0.68440
      slips   default 0.07664
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/86 [00:09<13:56,  9.84s/it]

skill_result:  0.653737432387006
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.55094
      learns  default 0.03256
      guesses default 0.47154
      slips   default 0.04801
      forgets default 0.00000


Fitting BKT per skill:   2%|█▍                                                          | 2/86 [00:20<14:36, 10.44s/it]

...

skill_result:  0.4425531914893617
skill:  85
✅ Parameters for skill: 85
                        value
skill param   class          
85    prior   default 0.77067
      learns  default 0.99742
      guesses default 0.35659
      slips   default 0.06592
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 85/86 [15:22<00:06,  6.87s/it]

skill_result:  0.5336879432624113
skill:  86
✅ Parameters for skill: 86
                        value
skill param   class          
86    prior   default 0.00117
      learns  default 0.17908
      guesses default 0.76304
      slips   default 0.00028
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 86/86 [16:07<00:00, 11.25s/it]

skill_result:  0.6547619047619048
mean_auc:  0.6377630837339162
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.68440 0.01967 0.38906 0.07664
1         1  0.00000  0.47154 0.03256 0.55094 0.04801
2        10  0.00000  0.02499 0.00781 0.92936 0.31330
3        11  0.00000  0.79775 0.00001 0.52663 0.05587
4        12  0.00000  0.27874 0.01138 0.22451 0.38903


In [7]:
run_BKT("Assistment_challenge_new")

Fitting BKT per skill:   0%|                                                                    | 0/26 [00:00<?, ?it/s]

skill:  0.0
✅ Parameters for skill: 0.0
                        value
skill param   class          
0.0   prior   default 0.57796
      learns  default 0.21626
      guesses default 0.04838
      slips   default 0.54278
      forgets default 0.00000


Fitting BKT per skill:   4%|██▎                                                         | 1/26 [00:11<04:37, 11.12s/it]

skill_result:  0.5342048826109391
skill:  1.0
✅ Parameters for skill: 1.0
                        value
skill param   class          
1.0   prior   default 0.97418
      learns  default 0.01896
      guesses default 0.05713
      slips   default 0.76312
      forgets default 0.00000


Fitting BKT per skill:   8%|████▌                                                       | 2/26 [01:19<17:54, 44.76s/it]

...

skill_result:  0.75
skill:  32.0
✅ Parameters for skill: 32.0
                        value
skill param   class          
32.0  prior   default 0.98265
      learns  default 0.99801
      guesses default 0.00000
      slips   default 0.54924
      forgets default 0.00000


Fitting BKT per skill:  96%|████████████████████████████████████████████████████████▋  | 25/26 [11:20<00:28, 28.90s/it]

skill_result:  0.5963963963963963
skill:  33.0
✅ Parameters for skill: 33.0
                        value
skill param   class          
33.0  prior   default 0.79329
      learns  default 0.00000
      guesses default 0.33667
      slips   default 0.26711
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 26/26 [11:32<00:00, 26.63s/it]

skill_result:  0.5074363992172212
mean_auc:  0.6026344661193871
param skill  forgets  guesses  learns   prior   slips
0       0.0  0.00000  0.04838 0.21626 0.57796 0.54278
1       1.0  0.00000  0.05713 0.01896 0.97418 0.76312
2      10.0  0.00000  0.29266 0.03109 0.26815 0.24868
3      11.0  0.00000  0.12181 0.17690 0.58553 0.40129
4      12.0  0.00000  0.11431 0.30649 0.21939 0.50257


In [14]:
poison_modes =[5,25,50]
dataset_names = ["HamptonAlg_new"] #,"Assistment_sequential_pattern"

for dataset_name in dataset_names:
    for poison_mode in poison_modes:
        run_BKT_dpa(dataset_name, "sequential_pattern", poison_mode)

0


Fitting BKT per skill:   0%|                                                                    | 0/86 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.34751
      learns  default 0.02001
      guesses default 0.68599
      slips   default 0.07859
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/86 [00:09<13:35,  9.59s/it]

skill_result:  0.6526754881887358
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.55043
      learns  default 0.03258
      guesses default 0.47162
      slips   default 0.04795
      forgets default 0.00000


...

skill_result:  0.4425531914893617
skill:  85
✅ Parameters for skill: 85
                        value
skill param   class          
85    prior   default 0.53671
      learns  default 0.98705
      guesses default 0.65179
      slips   default 0.06618
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 85/86 [16:28<00:07,  7.38s/it]

skill_result:  0.5115248226950355
skill:  86
✅ Parameters for skill: 86
                        value
skill param   class          
86    prior   default 0.60680
      learns  default 0.35492
      guesses default 0.47549
      slips   default 0.10658
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 86/86 [16:39<00:00, 11.62s/it]

skill_result:  0.6547619047619048
mean_auc:  0.6318901941768872
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.68089 0.01926 0.39791 0.08056
1         1  0.00000  0.47137 0.03254 0.55156 0.04809
2        10  0.00000  0.29430 0.00938 0.63750 0.20866
3        11  0.00000  0.79775 0.00001 0.52668 0.05591
4        12  0.00000  0.28046 0.01149 0.20729 0.38336


In [15]:
for poison_mode in poison_modes:
    run_BKT_dpa("HamptonAlg_new", "poisoned", poison_mode)

0


Fitting BKT per skill:   0%|                                                                    | 0/86 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.42588
      learns  default 0.01899
      guesses default 0.68254
      slips   default 0.07605
      forgets default 0.00000


Fitting BKT per skill:   1%|▋                                                           | 1/86 [00:09<13:03,  9.22s/it]

skill_result:  0.6541194267029307
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.56929
      learns  default 0.03456
      guesses default 0.46882
      slips   default 0.05049
      forgets default 0.00000


...

skill_result:  0.39680851063829786
skill:  85
✅ Parameters for skill: 85
                        value
skill param   class          
85    prior   default 0.59088
      learns  default 0.05908
      guesses default 0.48026
      slips   default 0.04699
      forgets default 0.00000


Fitting BKT per skill:  99%|██████████████████████████████████████████████████████████▎| 85/86 [20:59<00:14, 14.00s/it]

skill_result:  0.548758865248227
skill:  86
✅ Parameters for skill: 86
                        value
skill param   class          
86    prior   default 0.56668
      learns  default 0.02793
      guesses default 0.69059
      slips   default 0.08135
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 86/86 [21:14<00:00, 14.82s/it]

skill_result:  0.2976190476190476
mean_auc:  0.631361735809627
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.63292 0.04748 0.42649 0.08217
1         1  0.00000  0.39402 0.05069 0.68841 0.06883
2        10  0.00000  0.26208 0.01695 0.62768 0.17019
3        11  0.00000  0.76713 0.00000 0.60841 0.05910
4        12  0.00000  0.32399 0.01512 0.13945 0.23168


In [18]:
for poison_mode in poison_modes:
    print("Assistment_challenge_new_poisoned_",poison_mode)
    run_BKT_dpa("Assistment_new", "poisoned", poison_mode)

Assistment_challenge_new_poisoned_ 5
2


Fitting BKT per skill:   0%|                                                                    | 0/26 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.18031
      learns  default 0.10503
      guesses default 0.20831
      slips   default 0.32921
      forgets default 0.00000


Fitting BKT per skill:   4%|██▎                                                         | 1/26 [00:19<07:56, 19.05s/it]

skill_result:  0.5323988970588236
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.80124
      learns  default 0.00796
      guesses default 0.12211
      slips   default 0.69045
      forgets default 0.00000


...

skill_result:  0.75
skill:  32
✅ Parameters for skill: 32
                        value
skill param   class          
32    prior   default 0.57611
      learns  default 0.03926
      guesses default 0.35788
      slips   default 0.39615
      forgets default 0.00000


Fitting BKT per skill:  96%|████████████████████████████████████████████████████████▋  | 25/26 [12:00<00:22, 22.23s/it]

skill_result:  0.44054054054054054
skill:  33
✅ Parameters for skill: 33
                        value
skill param   class          
33    prior   default 0.26060
      learns  default 0.16641
      guesses default 0.46647
      slips   default 0.47305
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 26/26 [12:08<00:00, 28.02s/it]

skill_result:  0.508610567514677
mean_auc:  0.5332103501333516
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.55441 0.10165 0.18352 0.57178
1         1  0.00000  0.48127 0.24914 0.55444 0.50796
2        10  0.00000  0.55089 0.15627 0.30752 0.53358
3        11  0.00000  0.45770 0.44735 0.09013 0.39888
4        12  0.00000  0.45674 0.29621 0.72594 0.47827


In [19]:
for poison_mode in poison_modes:
    print("Assistment_challenge_new_poisoned_",poison_mode)
    run_BKT_dpa("Assistment_challenge_new", "sequential_pattern", poison_mode)

Assistment_challenge_new_poisoned_ 5
2


Fitting BKT per skill:   0%|                                                                    | 0/26 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.64756
      learns  default 0.29699
      guesses default 0.00076
      slips   default 0.57262
      forgets default 0.00000


Fitting BKT per skill:   4%|██▎                                                         | 1/26 [01:21<33:51, 81.25s/it]

skill_result:  0.5647292634158927
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.51707
      learns  default 0.02398
      guesses default 0.13560
      slips   default 0.65352
      forgets default 0.00000


...

skill_result:  0.75
skill:  32
✅ Parameters for skill: 32
                        value
skill param   class          
32    prior   default 0.98247
      learns  default 0.99795
      guesses default 0.00000
      slips   default 0.54920
      forgets default 0.00000


Fitting BKT per skill:  96%|████████████████████████████████████████████████████████▋  | 25/26 [14:02<00:31, 31.44s/it]

skill_result:  0.5963963963963963
skill:  33
✅ Parameters for skill: 33
                        value
skill param   class          
33    prior   default 0.78646
      learns  default 0.00001
      guesses default 0.33918
      slips   default 0.26579
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 26/26 [14:13<00:00, 32.81s/it]

skill_result:  0.5074363992172212
mean_auc:  0.6031568779303544
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.00058 0.29717 0.64817 0.57283
1         1  0.00000  0.07259 0.06643 0.80867 0.72538
2        10  0.00000  0.29251 0.02940 0.27410 0.25147
3        11  0.00000  0.12189 0.17685 0.58533 0.40120
4        12  0.00000  0.15368 0.23305 0.10321 0.43420


In [20]:
for poison_mode in poison_modes:
    print("Assistment_challenge_new_poisoned_",poison_mode)
    run_BKT_dpa("Assistment_challenge_new", "hint_abuse", poison_mode)

Assistment_challenge_new_poisoned_ 5
2


Fitting BKT per skill:   0%|                                                                    | 0/26 [00:00<?, ?it/s]

skill:  0
✅ Parameters for skill: 0
                        value
skill param   class          
0     prior   default 0.64550
      learns  default 0.28644
      guesses default 0.00542
      slips   default 0.57499
      forgets default 0.00000


Fitting BKT per skill:   4%|██▎                                                         | 1/26 [01:17<32:12, 77.29s/it]

skill_result:  0.5647292634158927
skill:  1
✅ Parameters for skill: 1
                        value
skill param   class          
1     prior   default 0.61211
      learns  default 0.03633
      guesses default 0.11743
      slips   default 0.67936
      forgets default 0.00000


...

skill_result:  0.75
skill:  32
✅ Parameters for skill: 32
                        value
skill param   class          
32    prior   default 0.63028
      learns  default 0.59579
      guesses default 0.07366
      slips   default 0.41215
      forgets default 0.00000


Fitting BKT per skill:  96%|████████████████████████████████████████████████████████▋  | 25/26 [09:42<00:12, 12.20s/it]

skill_result:  0.5972972972972972
skill:  33
✅ Parameters for skill: 33
                        value
skill param   class          
33    prior   default 0.54565
      learns  default 0.00001
      guesses default 0.37150
      slips   default 0.22253
      forgets default 0.00000


Fitting BKT per skill: 100%|███████████████████████████████████████████████████████████| 26/26 [09:53<00:00, 22.81s/it]

skill_result:  0.5054794520547945
mean_auc:  0.6006325586276765
param skill  forgets  guesses  learns   prior   slips
0         0  0.00000  0.00024 0.31012 0.70487 0.61307
1         1  0.00000  0.14468 0.01809 0.49216 0.67932
2        10  0.00000  0.24387 0.00742 0.31059 0.29229
3        11  0.00000  0.10409 0.18605 0.63248 0.42233
4        12  0.00000  0.00000 0.71219 0.63765 0.64763
